In [1]:
import os



In [4]:
!git clone -b jeremy --single-branch https://github.com/Jalalbaim/Direct-Preference-Optimization.git


Cloning into 'Direct-Preference-Optimization'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 160 (delta 76), reused 105 (delta 50), pack-reused 18 (from 1)
Receiving objects: 100% (160/160), 1.27 MiB | 17.10 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [3]:
rm -rf Direct-Preference-Optimization

In [ ]:
cd content/

In [5]:
# Changer de répertoire vers le projet cloné
%cd Direct-Preference-Optimization/


import os
print(f" Répertoire actuel: {os.getcwd()}")
print(f" Contenu: {os.listdir('.')[:10]}")

/content/Direct-Preference-Optimization
 Répertoire actuel: /content/Direct-Preference-Optimization
 Contenu: ['train_ppo.ipynb', 'configs', 'src', '.git', 'requirements.txt', '.gitignore', 'README.md', 'scripts', 'data']


In [6]:
%%capture
# Installation silencieuse des packages nécessaires
!pip install torch transformers datasets accelerate sentencepiece protobuf tqdm pyyaml scikit-learn

### Étape 2: Installation des dépendances

In [7]:

from google.colab import drive
drive.mount('/content/drive')
    
    

KeyboardInterrupt: 

## 1. Setup et Imports

In [8]:
import os
import sys
import torch
from torch.utils.data import DataLoader

# Ajouter le répertoire racine au path (déjà dans le bon répertoire après %cd)
ROOT = os.path.abspath('.')
if ROOT not in sys.path:
    sys.path.append(ROOT)

print(f" Working directory: {os.getcwd()}")

from src.dpo.models import load_models
from src.dpo.data import PromptDataset, prompt_collate_fn
from src.ppo.ppo_trainer import PPOTrainer
from src.dpo.utils import load_yaml_config

print("Imports réussis")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

 Working directory: /content/Direct-Preference-Optimization
Imports réussis
PyTorch version: 2.9.0+cu126
CUDA available: True


In [9]:
# Vérifier si les données de prompts existent
prompts_path = "data/processed/sentiment/prompts.jsonl"

if not os.path.exists(prompts_path):
    print(" Fichier prompts.jsonl non trouvé!")
    print("Exécution de prepare_prompts.py...")
    !python scripts/prepare_prompts.py
    print("Prompts préparés")
else:
    # Compter le nombre de prompts
    with open(prompts_path, 'r') as f:
        num_prompts = sum(1 for line in f if line.strip())
    print(f"{num_prompts} prompts trouvés dans {prompts_path}")

2497 prompts trouvés dans data/processed/sentiment/prompts.jsonl


## 4. Configuration des Hyperparamètres

Vous pouvez modifier ces paramètres selon vos besoins:

In [10]:
# Charger la config par défaut
config_path = "configs/ppo_sentiment.yaml"
config = load_yaml_config(config_path)

# Afficher les paramètres principaux
print(" Configuration PPO:")
print(f"  Model: {config['model']['name']}")
print(f"  Batch size: {config['training']['batch_size']}")
print(f"  Learning rate: {config['training']['learning_rate']}")
print(f"  Epochs: {config['training']['num_epochs']}")
print("\n Paramètres PPO:")
print(f"  Clip epsilon: {config['ppo']['clip_epsilon']}")
print(f"  Value coef: {config['ppo']['value_coef']}")
print(f"  Entropy coef: {config['ppo']['entropy_coef']}")
print(f"  Target KL: {config['ppo']['target_kl']}")
print(f"  PPO epochs: {config['ppo']['num_ppo_epochs']}")
print("\n Génération:")
print(f"  Max length: {config['generation']['max_length']}")
print(f"  Temperature: {config['generation']['temperature']}")

 Configuration PPO:
  Model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
  Batch size: 2
  Learning rate: 1e-5
  Epochs: 1

 Paramètres PPO:
  Clip epsilon: 0.2
  Value coef: 0.5
  Entropy coef: 0.01
  Target KL: 0.01
  PPO epochs: 4

 Génération:
  Max length: 128
  Temperature: 0.7


### Modifier les paramètres (optionnel)

Décommentez et modifiez si vous voulez changer certains paramètres:

In [11]:
# Exemple de modifications
# config['training']['batch_size'] = 1  # Réduire si problème de mémoire
# config['training']['num_epochs'] = 2  # Plus d'epochs
# config['ppo']['num_ppo_epochs'] = 2   # Moins d'epochs PPO par batch
# config['generation']['max_length'] = 64  # Réponses plus courtes

print("⚙️  Configuration personnalisée (si modifiée)")

⚙️  Configuration personnalisée (si modifiée)


## 5. Chargement des Modèles

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
model_name = config["model"]["name"]
dtype = config["model"]["dtype"]

# Charger les modèles (policy et référence)
mb = load_models(model_name, dtype=dtype, device=device)
tokenizer = mb.tokenizer

print(f" Modèles chargés: {model_name}")
print(f"   Policy model: {mb.policy_model.num_parameters():,} paramètres")
print(f"   Device: {mb.device}")

# 🔍 Diagnostic: Vérifier que tout est sur GPU
print(f"\n Diagnostic device:")
print(f"   Policy: {next(mb.policy_model.parameters()).device}")
print(f"   Ref: {next(mb.ref_model.parameters()).device}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


 Modèles chargés: TinyLlama/TinyLlama-1.1B-Chat-v1.0
   Policy model: 1,100,048,384 paramètres
   Device: cuda

 Diagnostic device:
   Policy: cuda:0
   Ref: cuda:0


## 6. Préparation du DataLoader

In [14]:
# Charger le dataset de prompts
prompt_dataset = PromptDataset(config["data"]["prompt_path"])
max_prompt_length = config["data"]["max_prompt_length"]

print(f"Dataset chargé: {len(prompt_dataset)} prompts")

# Fonction de collate
def collate(batch):
    return prompt_collate_fn(
        batch,
        tokenizer=tokenizer,
        max_prompt_length=max_prompt_length,
    )

# DataLoader
prompt_loader = DataLoader(
    prompt_dataset,
    batch_size=config["training"]["batch_size"],
    shuffle=True,
    collate_fn=collate,
)

print(f" DataLoader créé: {len(prompt_loader)} batches")

Dataset chargé: 2497 prompts
 DataLoader créé: 1249 batches


## 7. Initialisation du PPO Trainer

In [15]:
print("⚙️ Initialisation du PPO Trainer...")

# Créer le trainer
trainer = PPOTrainer(
    model_bundle=mb,
    prompt_loader=prompt_loader,
    config=config,
)

print("✅ Trainer initialisé")
print(f"   Reward model: {config['reward_model']['name']}")
print(f"   Save dir: {config['logging']['save_dir']}")

# 🔍 Diagnostic: Vérifier que tout est sur le même device
print(f"\n🔍 Diagnostic final - Tous les composants:")
print(f"   Trainer device: {trainer.device}")
print(f"   Policy model: {next(trainer.policy_model.parameters()).device}")
print(f"   Value head: {next(trainer.policy_model.value_head.parameters()).device}")
print(f"   Ref model: {next(trainer.ref_model.parameters()).device}")
print(f"   Reward model: {trainer.reward_model.device}")

⚙️ Initialisation du PPO Trainer...


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

✅ Trainer initialisé
   Reward model: lvwerra/distilbert-imdb
   Save dir: checkpoints/sentiment_ppo

🔍 Diagnostic final - Tous les composants:
   Trainer device: cuda
   Policy model: cuda:0
   Value head: cuda:0
   Ref model: cuda:0
   Reward model: cuda


## 8. Entraînement PPO 

In [16]:
# 🔍 Système de diagnostic pour tracer les erreurs
def debug_tensor(name, tensor, indent=0):
    """Affiche les infos détaillées d'un tenseur"""
    prefix = "  " * indent
    if tensor is None:
        print(f"{prefix}{name}: None")
    elif isinstance(tensor, torch.Tensor):
        print(f"{prefix}{name}: shape={tensor.shape}, dtype={tensor.dtype}, device={tensor.device}, requires_grad={tensor.requires_grad}")
    elif isinstance(tensor, (list, tuple)):
        print(f"{prefix}{name}: {type(tensor).__name__} of length {len(tensor)}")
        if len(tensor) > 0 and isinstance(tensor[0], torch.Tensor):
            debug_tensor(f"{name}[0]", tensor[0], indent+1)
    elif isinstance(tensor, dict):
        print(f"{prefix}{name}: dict with keys {list(tensor.keys())}")
        for k, v in tensor.items():
            debug_tensor(f"{name}['{k}']", v, indent+1)
    else:
        print(f"{prefix}{name}: {type(tensor).__name__} = {tensor}")

# Wrapper pour le _ppo_step avec diagnostic
original_ppo_step = trainer._ppo_step

def debug_ppo_step(batch):
    print("\n" + "="*80)
    print("🔍 DEBUG PPO STEP - DÉBUT")
    print("="*80)
    
    try:
        print("\n1️⃣ BATCH INPUT:")
        debug_tensor("batch", batch)
        
        # Vérifier que batch est déplacé sur device
        batch_on_device = {k: v.to(trainer.device) for k, v in batch.items()}
        print("\n2️⃣ BATCH APRÈS .to(device):")
        debug_tensor("batch_on_device", batch_on_device)
        
        print("\n3️⃣ GÉNÉRATION DES RÉPONSES...")
        trainer.policy_model.eval()
        
        # Vérifier les inputs de génération
        print("\n  Inputs de génération:")
        debug_tensor("  prompt_ids", batch_on_device["input_ids"])
        debug_tensor("  prompt_mask", batch_on_device["attention_mask"])
        
        # Génération
        generated_ids = trainer.policy_model.generate(
            input_ids=batch_on_device["input_ids"],
            attention_mask=batch_on_device["attention_mask"],
            max_new_tokens=trainer.max_gen_length,
            temperature=trainer.temperature,
            do_sample=True,
            pad_token_id=trainer.mb.tokenizer.pad_token_id,
        )
        print("\n  Outputs de génération:")
        debug_tensor("  generated_ids", generated_ids)
        
        # Créer response_mask
        prompt_len = batch_on_device["input_ids"].shape[1]
        model_dtype = next(trainer.policy_model.parameters()).dtype
        response_mask = torch.zeros_like(generated_ids, dtype=model_dtype)
        response_mask[:, prompt_len:] = 1
        
        print("\n  Masques créés:")
        debug_tensor("  response_mask", response_mask)
        
        # Calculer logits
        full_mask = torch.ones_like(generated_ids, dtype=torch.long)
        debug_tensor("  full_mask", full_mask)
        
        print("\n4️⃣ FORWARD POLICY MODEL...")
        policy_outputs = trainer.policy_model(
            input_ids=generated_ids,
            attention_mask=full_mask,
        )
        print("  Policy outputs:")
        debug_tensor("  logits", policy_outputs.logits)
        
        print("\n5️⃣ CALCUL DES LOG-PROBS...")
        shift_logits = policy_outputs.logits[:, :-1, :].contiguous()
        shift_labels = generated_ids[:, 1:].contiguous()
        shift_mask = response_mask[:, 1:].contiguous()
        
        print("  Tenseurs décalés:")
        debug_tensor("  shift_logits", shift_logits)
        debug_tensor("  shift_labels", shift_labels)
        debug_tensor("  shift_mask", shift_mask)
        
        log_probs = torch.log_softmax(shift_logits, dim=-1)
        debug_tensor("  log_probs", log_probs)
        
        token_logps = log_probs.gather(-1, shift_labels.unsqueeze(-1)).squeeze(-1)
        debug_tensor("  token_logps", token_logps)
        
        token_logps_masked = token_logps * shift_mask
        debug_tensor("  token_logps_masked", token_logps_masked)
        
        seq_logps = token_logps_masked.sum(dim=-1)
        debug_tensor("  seq_logps", seq_logps)
        
        print("\n6️⃣ CALCUL DES REWARDS...")
        texts = trainer.mb.tokenizer.batch_decode(generated_ids[:, prompt_len:], skip_special_tokens=True)
        print(f"  Textes générés: {texts}")
        
        rewards = trainer.reward_model.compute_rewards(texts)
        debug_tensor("  rewards (avant conversion)", rewards)
        
        rewards = rewards.to(dtype=model_dtype)
        debug_tensor("  rewards (après conversion)", rewards)
        
        print("\n7️⃣ FORWARD AVEC HIDDEN STATES...")
        old_outputs = trainer.policy_model(
            input_ids=generated_ids,
            attention_mask=full_mask,
            output_hidden_states=True,
        )
        print("  Hidden states:")
        if hasattr(old_outputs, 'hidden_states') and old_outputs.hidden_states:
            debug_tensor("  hidden_states[-1]", old_outputs.hidden_states[-1])
            last_hidden = old_outputs.hidden_states[-1][:, -1, :]
            debug_tensor("  last_hidden", last_hidden)
            
            print("\n8️⃣ VALUE HEAD...")
            values = trainer.policy_model.value_head(last_hidden)
            debug_tensor("  values", values)
            
            print("\n9️⃣ ADVANTAGES...")
            advantages = rewards - values
            debug_tensor("  advantages", advantages)
        
        print("\n✅ APPEL DE LA FONCTION ORIGINALE...")
        trainer.policy_model.train()
        
    except Exception as e:
        print("\n" + "="*80)
        print(f"❌ ERREUR DÉTECTÉE: {type(e).__name__}")
        print(f"   Message: {e}")
        print("="*80)
        import traceback
        traceback.print_exc()
        raise
    
    # Appeler la fonction originale
    return original_ppo_step(batch_on_device)

# Remplacer temporairement _ppo_step par la version debug
trainer._ppo_step = debug_ppo_step

print("✅ Système de diagnostic activé")
print("   Chaque step PPO affichera les types/devices/shapes de tous les tenseurs")

✅ Système de diagnostic activé
   Chaque step PPO affichera les types/devices/shapes de tous les tenseurs


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [17]:
import time

print("="*60)
print(f"🚀 DÉMARRAGE DE L'ENTRAÎNEMENT PPO (AVEC DIAGNOSTIC)")
print(f"   Expérience: {config['experiment_name']}")
print(f"   Device: {device}")
print("="*60)

start_time = time.time()

try:
    # Lancer l'entraînement avec diagnostic
    trainer.train()
    
    elapsed = time.time() - start_time
    print("\n" + "="*60)
    print(f"✅ Entraînement terminé!")
    print(f"   Temps total: {elapsed/60:.2f} minutes")
    print("="*60)
    
except KeyboardInterrupt:
    print("\n⚠️  Entraînement interrompu par l'utilisateur")
    print("   Les checkpoints partiels ont été sauvegardés")
    
except Exception as e:
    print("\n" + "="*80)
    print(f"❌ ERREUR FINALE: {type(e).__name__}")
    print(f"   Message: {e}")
    print("="*80)
    import traceback
    traceback.print_exc()
    print("\n💡 Les détails de l'erreur sont affichés ci-dessus dans les logs de debug")

🚀 DÉMARRAGE DE L'ENTRAÎNEMENT PPO (AVEC DIAGNOSTIC)
   Expérience: ppo_sentiment_tinyllama
   Device: cuda


PPO Epoch 1/1:   0%|          | 0/1249 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



🔍 DEBUG PPO STEP - DÉBUT

1️⃣ BATCH INPUT:
batch: dict with keys ['input_ids', 'attention_mask']
  batch['input_ids']: shape=torch.Size([2, 76]), dtype=torch.int64, device=cpu, requires_grad=False
  batch['attention_mask']: shape=torch.Size([2, 76]), dtype=torch.int64, device=cpu, requires_grad=False

2️⃣ BATCH APRÈS .to(device):
batch_on_device: dict with keys ['input_ids', 'attention_mask']
  batch_on_device['input_ids']: shape=torch.Size([2, 76]), dtype=torch.int64, device=cuda:0, requires_grad=False
  batch_on_device['attention_mask']: shape=torch.Size([2, 76]), dtype=torch.int64, device=cuda:0, requires_grad=False

3️⃣ GÉNÉRATION DES RÉPONSES...

  Inputs de génération:
  prompt_ids: shape=torch.Size([2, 76]), dtype=torch.int64, device=cuda:0, requires_grad=False
  prompt_mask: shape=torch.Size([2, 76]), dtype=torch.int64, device=cuda:0, requires_grad=False

  Outputs de génération:
  generated_ids: shape=torch.Size([2, 204]), dtype=torch.int64, device=cuda:0, requires_grad=False

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


  values: shape=torch.Size([2]), dtype=torch.bfloat16, device=cuda:0, requires_grad=True

9️⃣ ADVANTAGES...
  advantages: shape=torch.Size([2]), dtype=torch.bfloat16, device=cuda:0, requires_grad=True

✅ APPEL DE LA FONCTION ORIGINALE...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



❌ ERREUR FINALE: OutOfMemoryError
   Message: CUDA out of memory. Tried to allocate 22.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 56887 has 14.73 GiB memory in use. Of the allocated memory 14.18 GiB is allocated by PyTorch, and 430.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

💡 Les détails de l'erreur sont affichés ci-dessus dans les logs de debug


Traceback (most recent call last):
  File "/tmp/ipython-input-3226112305.py", line 13, in <cell line: 0>
    trainer.train()
  File "/content/Direct-Preference-Optimization/src/ppo/ppo_trainer.py", line 85, in train
    stats = self._ppo_step(batch)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-145877373.py", line 143, in debug_ppo_step
    return original_ppo_step(batch_on_device)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/Direct-Preference-Optimization/src/ppo/ppo_trainer.py", line 210, in _ppo_step
    self.optimizer.step()
  File "/usr/local/lib/python3.12/dist-packages/torch/optim/optimizer.py", line 517, in wrapper
    out = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/optim/optimizer.py", line 82, in _use_grad
    ret = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/optim/adam.py", line 247, in step
    adam(
  File "/usr/

## ⚠️ FIX APPLIQUÉ - REDÉMARRER LE KERNEL

**Problème identifié :** Le `ValueHead` était créé avec `dtype=torch.bfloat16` en dur, ce qui causait un mismatch avec le modèle.

**Solution :** Le code a été corrigé dans [src/dpo/reward_models.py](src/dpo/reward_models.py).

**Action requise :** 
1. ⚡ **Redémarrer le kernel** (pour recharger le code Python modifié)
2. 🔄 **Réexécuter les cellules** depuis le début (Setup, Imports, Config, Modèles, Trainer)
3. ▶️ **Relancer l'entraînement**

## 9. Vérification des Checkpoints

In [ ]:
import glob

save_dir = config['logging']['save_dir']
checkpoints = glob.glob(os.path.join(save_dir, "*.pt"))

if checkpoints:
    print(f"✅ {len(checkpoints)} checkpoint(s) sauvegardé(s):")
    for ckpt in sorted(checkpoints):
        size_mb = os.path.getsize(ckpt) / (1024**2)
        print(f"   📁 {os.path.basename(ckpt)} ({size_mb:.1f} MB)")
else:
    print("⚠️  Aucun checkpoint trouvé")

## 10. Test de Génération Rapide

In [ ]:
# Tester la génération avec le modèle entraîné
test_prompts = [
    "The movie was",
    "I think this product is",
    "The customer service was",
]

print("🧪 Test de génération avec le modèle PPO:")
print("="*60)

mb.policy_model.eval()
with torch.no_grad():
    for prompt in test_prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        outputs = mb.policy_model.generate(
            **inputs,
            max_new_tokens=32,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\n💬 Prompt: {prompt}")
        print(f"✨ Response: {response}")

print("\n" + "="*60)

## 🎯 Prochaines Étapes

Maintenant que l'entraînement est terminé, vous pouvez:

1. **Évaluer le modèle** avec `eval_sentiment2.py`:
   ```bash
   python scripts/eval_sentiment2.py --method ppo --num_samples 3
   ```

2. **Comparer avec DPO et GRPO**:
   ```bash
   python scripts/compare_methods.py
   ```

3. **Visualiser les résultats**:
   ```bash
   python scripts/visualize_comparison.py
   ```

4. **Modifier les hyperparamètres** et réentraîner pour comparer les performances

## 📊 Notes

- Les checkpoints sont sauvegardés dans `checkpoints/sentiment_ppo/`
- Le modèle PPO inclut un **value head** en plus de la policy
- L'entraînement utilise un **reward model** basé sur le sentiment
- Le **target KL** permet l'early stopping pour la stabilité
- Sur CPU, l'entraînement est plus lent mais plus stable que sur MPS (macOS)